# Introdução

*Script* que cria uma tabela que apresenta informações das *Coordenadorias Regionais de Proteção e Defesa Civil - **REPDEC*** da Defesa Civil que atuam nos 645 município deo Estado de São Paulo.

In [ ]:
#!pip3 install PyPDF2 --upgrade
!pip3 install pdf2jpg --upgrade

In [ ]:
import os
import random
import shutil
import time
import urllib.parse
from zipfile import ZipFile

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from osgeo import gdal, ogr, osr
from pdf2jpg import pdf2jpg
from PyPDF2 import PdfFileReader, PdfFileWriter
from tqdm.notebook import tqdm, trange

# Dados Espaciais: PDF

## *Download*

Inicialmente foi feito o download da [monografia](http://www.sidec.sp.gov.br/defesacivil/media/OSDownloads/1398863063_Monografia%20Cap%20PM%20Rudyard.pdf)  desenvolvida pelo Capitão da PM, Rudyard Panzarini Paiva, em 2013, com o título *'Regulamentação dos Critérios para Homologação das Situações de Anormalidade no Estado de São Paulo'*, elaborado para concluir o 'Curso de Aperfeiçoamento de Oficiais – II/2012'.

Na monografia, na página 33, é apresentado o mapa com as áreas de abrangência de cada que Indicações sobre a área de atuação de cada Coordenadorias Regionais de Proteção e Defesa Civil - REPDEC, além disso é informado que:


> As jurisdições das Coordenadorias Regionais de Proteção e Defesa Civil acompanham as Regiões Administrativas do Estado de São Paulo, porém majoradas para 19, pois foram acrescentadas as regiões do ABCDMRR, Guarulhos e Osasco.

<br>

Pesquisando informaçãoes sobre a sigla mencionada, encontrou que:

> ABC Paulista, Região do Grande ABC, ABC ou ainda ABCD, é uma região tradicionalmente industrial do estado de São Paulo, parte da Região Metropolitana de São Paulo, porém com identidade própria. A sigla vem das três cidades que, originalmente, formavam a região, sendo: Santo André (A), São Bernardo do Campo (B) e São Caetano do Sul (C). Às vezes, Diadema (D) é incluída na sigla.

> É relativamente comum encontrar também ABCDMRR que também inclui os municípios de Mauá, Ribeirão Pires e Rio Grande da Serra.


In [ ]:
def download_urls(urls, path, GetFilenameFromURL=True):
    """
    Function to download list of files with a progress bar.
    :param urls: Lista ou tuple, a depender da função #
    :param path: Local onde os arquivos serão inseridos
    :param GetFilenameFromURL: Se VERDADEIRO, a função irá nomear o arquivo conforme o link das lista das URLs.
    Se FALSO, a função irá nomear o arquivo conforme string definida no tuple das URLs
    """
    
    # Reset Interactions
    i = 0
    n_urls = len(urls)

    for n_url in trange(n_urls, desc='Total'):
        # Download path and file name
        if GetFilenameFromURL:
            url = urls[i]
            filename = urls[i].rsplit('/', 1)[1]
        else:
            url = urls[i][0]
            filename = urls[i][1]

        # File size
        r = requests.get(url, stream=True)
        chunk_size = 1024 * 1024
        total_size = int(r.headers['content-length'])

        # Download the file from 'url' and save it locally under 'filename'
        with open(os.path.join(path, filename), 'wb') as f:
            for data in tqdm(
                iterable = r.iter_content(chunk_size=chunk_size),
                total = int(total_size/chunk_size),
                unit = 'MB',
                desc = '{}/{}'.format(str(i+1), str(n_urls))):
                f.write(data)

        # Interactions
        i = i+1

        # Definir um intervalo de tempo
        time.sleep(random.randint(1, 3))

In [ ]:
url = 'http://www.sidec.sp.gov.br/defesacivil/media/OSDownloads/1398863063_Monografia%20Cap%20PM%20Rudyard.pdf'
filename= '2013 - Regulamentação dos critérios.pdf'
tuple_pdf = [(url, filename)]

In [ ]:
download_urls(tuple_pdf, 'docs', GetFilenameFromURL=False)

<br>

## Recortar mapa do PDF

Uma vez com o arquivo *pdf*, selecionei apenas a página que me interessava.

In [ ]:
# I/O
in_filename = os.path.join('docs', '2013 - Regulamentação dos critérios.pdf')
out_filename = os.path.join('docs', 'temp.pdf')

# Read File
in_pdf = PdfFileReader(in_filename)

# Select Pages
pages = [33]

# Create Object to output
out_pdf = PdfFileWriter()

# Loop trough pages
for page in pages:
    out_pdf.addPage(in_pdf.getPage(page))

# Save
with open(out_filename, 'wb') as f:
    out_pdf.write(f)
    f.close()

<br>

E recortei o trecho que me interessava

In [ ]:
# I/O
in_filename = os.path.join('docs', 'temp.pdf')
out_filename = os.path.join('docs', 'mapa.pdf')

with open(in_filename, 'rb') as f:
    in_pdf = PdfFileReader(f)
    out_pdf = PdfFileWriter()

    numPages = in_pdf.getNumPages()
    print('document has {} pages.'.format(numPages))

    for i in range(numPages):
        page = in_pdf.getPage(i)
        print(
            page.mediaBox.getUpperRight_x(),
            page.mediaBox.getUpperRight_y()
        )        
        #page.trimBox.lowerLeft = (0, 0)
        #page.trimBox.upperRight = (525, 625)
        page.cropBox.lowerLeft = (130, 380)
        page.cropBox.upperRight = (530, 700)
        out_pdf.addPage(page)

    with open(out_filename, 'wb') as out_f:
        out_pdf.write(out_f)

<br>

Posteriormente converti para imagem.

In [ ]:
# to convert all pages
result = pdf2jpg.convert_pdf2jpg(
    os.path.join('docs', 'mapa.pdf'),
    os.path.join('docs'),
    pages='ALL')

print(result)

<br>

Renomeie

In [ ]:
os.rename(
    result[0]['output_jpgfiles'][0],
    os.path.join('data', 'rasters', 'mapa.jpg')    
)

<br>

E deletei os arquivos temporários.

In [ ]:
os.remove(in_filename)
os.remove(out_filename)
shutil.rmtree(os.path.dirname(result[0]['output_jpgfiles'][0]))

<br>

## Georeferenciamento

Uma vez com o mapa em mãos, em *jpg*, fiz o georreferenciamento dele.

In [ ]:
def georeferencing(src_filename, dst_filename, gcp_list):
    # 
    src_ds = gdal.Open(src_filename, gdal.GA_ReadOnly)
    driver = gdal.GetDriverByName('JPEG')

    # Open destination dataset
    dst_ds = driver.CreateCopy(dst_filename, src_ds, 0)

    # Get raster projection
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4674)
    dst_wkt = srs.ExportToWkt()

    # Apply the GCPs to the open output file:
    dst_ds.SetGCPs(gcp_list, dst_wkt)

    # Warp Image
    dst_ds = gdal.Warp(dst_filename, dst_ds, dstSRS=dst_wkt)

    # Close files
    src_ds = None
    dst_ds = None
    
    print('Arquivo "{}" georreferenciado.'.format(src_filename))

In [ ]:
# Interior
gcp = []
gcp.append(gdal.GCP(-53.08776489000207732,-22.65848512212706112,0,67.39315671031094723,734.81352291325697479))
gcp.append(gdal.GCP(-44.1613485118776623,-22.67839016588562018,0,1613.78769435353024164,693.92426605975663279))
gcp.append(gdal.GCP(-48.89569728207362687,-20.44167357629228476,0,784.82150163667267861,196.14571399347153147))
gcp.append(gdal.GCP(-48.09945589998689996,-25.31233573226733213,0,943.74248158757416149,1298.82619425124539703))
gcp.append(gdal.GCP(-46.64917183567540349,-21.36317108739623194,0,1179.87410495091262419,396.53545673078855316))
gcp.append(gdal.GCP(-51.77248636389690262,-21.05130888861864591,0,284.51187218700169979,359.08062474429112854))

georeferencing(
    os.path.join('data', 'rasters', 'mapa.jpg'),
    os.path.join('data', 'rasters', 'geo_defesacivil.jpg'),
    gcp,
)

<br>

## Spatial Join

Usando o código *get_spatial_data.ipynb* foi possível baixar a malha municipal do estado de São Paulo do [*site* do IBGE](https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2020/UFs/SP/SP_Municipios_2020.zip). Nos códigos abaixo o arquivo *shapefile* é lido, são feitos pequenos ajustes para ter um material mais "limpo" para definir, manualmente, quais os "Grupamentos de Bombeiro" que atual em cada um dos municípios.

In [ ]:
# Lê o arquivo shapefile
file = 'sp_ibge_250k'
shp = os.path.join('data', 'shps', '{}.shp'.format(file))
gdf = gpd.read_file(shp)

# Deleta colunas
gdf.drop(['SIGLA_UF', 'AREA_KM2'], axis=1, inplace=True)

# Renomeia colunas
gdf.rename(columns={'CD_MUN':'id_mun'}, inplace=True)
gdf.rename(columns={'NM_MUN':'municipio'}, inplace=True)

# Reordena colunas
gdf['repdec_cod'] = 'I'
gdf = gdf[['id_mun', 'municipio', 'repdec_cod', 'geometry']]

# ATENÇÃO: SÓ EXECUTAR SE FOR REFAZER A CLASSIFICAÇÃO MANUALMENTE NO QGIS!
#gdf.to_file(os.path.join('data', 'shps', 'div_admin.shp'), encoding='utf-8')

# Results
gdf.head()